In [1]:
import numpy as np
import pandas as pd
import pickle
import xgboost
import os
from scipy.stats import ttest_1samp, ttest_ind, ttest_rel
from config import path_prefix_results
import warnings

warnings.filterwarnings('ignore')


## Create latex template for leg, loin, belly, shoulder, body results

In [2]:
#latex output setup
latex_template = r'''\documentclass[a4paper]{{article}}
\usepackage{{booktabs}}
\usepackage{{pdflscape}}
\begin{{document}}
\thispagestyle{{empty}}
\begin{{landscape}}
{}
\end{{landscape}}
\end{{document}}
'''

path_leg_r2_latex= os.path.join(path_prefix_results, 'leg_r2.tex')
path_leg_rmse_latex= os.path.join(path_prefix_results, 'leg_rmse.tex')
path_leg_ttest_latex= os.path.join(path_prefix_results, 'leg_ttest.tex')
path_leg_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'leg_fs_ttest_diff.tex')
path_leg_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'leg_fs_ttest_diff_cross.tex')

path_loin_r2_latex= os.path.join(path_prefix_results, 'loin_r2.tex')
path_loin_rmse_latex= os.path.join(path_prefix_results, 'loin_rmse.tex')
path_loin_ttest_latex= os.path.join(path_prefix_results, 'loin_ttest.tex')
path_loin_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'loin_fs_ttest_diff.tex')
path_loin_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'loin_fs_ttest_diff_cross.tex')

path_belly_r2_latex= os.path.join(path_prefix_results, 'belly_r2.tex')
path_belly_rmse_latex= os.path.join(path_prefix_results, 'belly_rmse.tex')
path_belly_ttest_latex= os.path.join(path_prefix_results, 'belly_ttest.tex')
path_belly_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'belly_fs_ttest_diff.tex')
path_belly_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'belly_fs_ttest_diff_cross.tex')

path_shoulder_r2_latex= os.path.join(path_prefix_results, 'shoulder_r2.tex')
path_shoulder_rmse_latex= os.path.join(path_prefix_results, 'shoulder_rmse.tex')
path_shoulder_ttest_latex= os.path.join(path_prefix_results, 'shoulder_ttest.tex')
path_shoulder_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'shoulder_fs_ttest_diff.tex')
path_shoulder_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'shoulder_fs_ttest_diff_cross.tex')

path_body_r2_latex= os.path.join(path_prefix_results, 'body_r2.tex')
path_body_rmse_latex= os.path.join(path_prefix_results, 'body_rmse.tex')
path_body_ttest_latex= os.path.join(path_prefix_results, 'body_ttest.tex')
path_body_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'body_fs_ttest_diff.tex')
path_body_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'body_fs_ttest_diff_cross.tex')

In [3]:
def test_diff_cross(df, row='multi-atlas'):
    tmp= pd.DataFrame(index=df.columns, columns=df.columns)
    for i in df.columns:
        for j in df.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df.loc[row, i]), np.abs(df.loc[row, j]))[1]
    return tmp

def test_diff(df, col='multi-atlas'):
    tmp= pd.DataFrame(index=df.index, columns=df.columns)
    for i in df.index:
        for j in df.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df.loc[i, j]), np.abs(df.loc[col, j]))[1]
    tmp= tmp.drop(col, axis='rows')
    return tmp

def test_compare(df0, df1):
    tmp= pd.DataFrame(index=df0.index, columns=df0.columns)
    for i in df0.index:
        for j in df0.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df0.loc[i, j]), np.abs(df1.loc[i, j]))[1]
    return tmp

## Load data 

In [4]:
data_fs= pickle.load(open(os.path.join(path_prefix_results,'results.pickle'), 'rb'))
data_no_fs= pickle.load(open(os.path.join(path_prefix_results,'results_no_fs.pickle'), 'rb'))

In [5]:
leg_fs= data_fs[data_fs['dataset'] == 'leg']
loin_fs= data_fs[data_fs['dataset'] == 'loin']
belly_fs= data_fs[data_fs['dataset'] == 'belly']
shoulder_fs= data_fs[data_fs['dataset'] == 'shoulder']
body_fs= data_fs[data_fs['dataset'] == 'body']

leg_no_fs= data_no_fs[data_no_fs['dataset'] == 'leg']
loin_no_fs= data_no_fs[data_no_fs['dataset'] == 'loin']
belly_no_fs= data_no_fs[data_no_fs['dataset'] == 'belly']
shoulder_no_fs= data_no_fs[data_no_fs['dataset'] == 'shoulder']
body_no_fs= data_no_fs[data_no_fs['dataset'] == 'body']

In [6]:
rename_dict_regr= {'KNNR_Objective': 'kNN', 'LinearRegression_Objective': 'linear', 'PLSRegression_Objective': 'PLS', 'RidgeRegression_Objective': 'ridge', 'LassoRegression_Objective': 'lasso'}
#rename_dict_type= {'203a': 'atlas 1', '203k': 'atlas 2', '204f': 'atlas 3', '206k': 'atlas 4', '208f': 'atlas 5', 'all': 'multi-atlas', 'mean_mask': 'mean'}
#row_order= ['atlas 1', 'atlas 2', 'atlas 3', 'atlas 4', 'atlas 5', 'mean', 'multi-atlas']
rename_dict_type= sorted(leg_fs['type'].unique())
rename_dict_type= dict(map(lambda x, i: (x, f"atlas {i+1}"), rename_dict_type,  range(len(rename_dict_type))))
rename_dict_type['all']= "multi-atlas"
rename_dict_type['mean_mask'] = "mean"
print(rename_dict_type)
row_order= list(rename_dict_type.values())
col_order= ['linear', 'PLS', 'lasso', 'ridge', 'kNN']

{'all': 'multi-atlas', 'mean_mask': 'mean', 'p004': 'atlas 3', 'p029': 'atlas 4', 'p032': 'atlas 5', 'p041': 'atlas 6', 'p048': 'atlas 7'}


## leg $r^2$ results

In [7]:
leg_fs_results= leg_fs.pivot(index='type', columns='model', values='r2_1')
leg_fs_results= leg_fs_results.rename(rename_dict_regr, axis='columns')
leg_fs_results= leg_fs_results.rename(rename_dict_type)
leg_fs_results= leg_fs_results.apply(lambda x: np.round(x, 4))
leg_fs_results= leg_fs_results.loc[row_order, col_order]

leg_no_fs_results= leg_no_fs.pivot(index='type', columns='model', values='r2_1')
leg_no_fs_results= leg_no_fs_results.rename(rename_dict_regr, axis='columns')
leg_no_fs_results= leg_no_fs_results.rename(rename_dict_type)
leg_no_fs_results= leg_no_fs_results.apply(lambda x: np.round(x, 4))
leg_no_fs_results= leg_no_fs_results.loc[row_order, col_order]

leg_all= pd.concat([leg_fs_results, leg_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
leg_all.columns= columns

In [8]:
leg_all_r2_latex= leg_all.to_latex()
print(leg_all_r2_latex)

#save results to tex file
with open(path_leg_r2_latex, 'w') as f:
    f.write(latex_template.format(leg_all_r2_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
multi-atlas & 0.987400 & 0.984600 & 0.987900 & 0.985800 & 0.922500 & 0.914300 & 0.960400 & 0.968800 & 0.954600 & 0.766800 \\
mean & 0.975700 & 0.975900 & 0.975700 & 0.975300 & 0.905100 & 0.923700 & 0.958600 & 0.966200 & 0.963200 & 0.782100 \\
atlas 3 & 0.977300 & 0.975300 & 0.978000 & 0.978200 & 0.923900 & 0.929500 & 0.957400 & 0.968500 & 0.965800 & 0.779600 \\
atlas 4 & 0.968700 & 0.969900 & 0.968600 & 0.967700 & 0.893200 & 0.907400 & 0.957000 & 0.957900 & 0.957300 & 0.745600 \\
atlas 5 & 0.971900 & 0.970100 & 0.971800 & 0.971600 & 0.907400 & 0.913900 & 0.951500 & 0.963400 & 0.958500 & 0.706900 \\
atlas 6 & 0.977900 & 0.970100 & 0.974000 & 0.974800 & 0.891400 & 0.919600 & 0.953900 & 0.963800 & 0.963300 & 0.715400 \\
atlas 7 & 0.974600 & 0.

## leg RMSE results

In [9]:
leg_fs_results_rmse= leg_fs.pivot(index='type', columns='model', values='rmse_1')
leg_fs_results_rmse= leg_fs_results_rmse.rename(rename_dict_regr, axis='columns')
leg_fs_results_rmse= leg_fs_results_rmse.rename(rename_dict_type)
leg_fs_results_rmse= leg_fs_results_rmse.apply(lambda x: np.round(x, 4))
leg_fs_results_rmse= leg_fs_results_rmse.loc[row_order, col_order]

leg_no_fs_results_rmse= leg_no_fs.pivot(index='type', columns='model', values='rmse_1')
leg_no_fs_results_rmse= leg_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
leg_no_fs_results_rmse= leg_no_fs_results_rmse.rename(rename_dict_type)
leg_no_fs_results_rmse= leg_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
leg_no_fs_results_rmse= leg_no_fs_results_rmse.loc[row_order, col_order]

leg_all_rmse= pd.concat([leg_fs_results_rmse, leg_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
leg_all_rmse.columns= columns

In [10]:
leg_all_rmse_latex= leg_all_rmse.to_latex()
print(leg_all_rmse_latex)

#save results to tex file
with open(path_leg_rmse_latex, 'w') as f:
    f.write(latex_template.format(leg_all_rmse_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
multi-atlas & 136.948000 & 151.591000 & 134.293400 & 145.761000 & 339.954600 & 357.446800 & 243.118300 & 215.632600 & 260.181800 & 589.689800 \\
mean & 190.477800 & 189.395600 & 190.419900 & 191.882100 & 376.169900 & 337.351700 & 248.516500 & 224.547800 & 234.279400 & 570.032200 \\
atlas 3 & 183.906600 & 191.794300 & 180.988900 & 180.225100 & 336.866900 & 324.299500 & 251.986800 & 216.643200 & 225.975200 & 573.322400 \\
atlas 4 & 216.017100 & 211.966100 & 216.495800 & 219.414600 & 399.014000 & 371.602900 & 253.104000 & 250.449400 & 252.372400 & 615.849200 \\
atlas 5 & 204.820900 & 211.289900 & 204.933200 & 205.756900 & 371.540500 & 358.374600 & 268.965400 & 233.733200 & 248.735400 & 661.070800 \\
atlas 6 & 181.663200 & 211.213600 & 196.7236

## t-test leg

In [11]:
leg_fs['diff']= (leg_fs['y_pred_1'] - leg_fs['y_test_1'])**2
leg_fs_test= leg_fs.pivot(index='type', columns='model', values='diff')
leg_fs_test= leg_fs_test.rename(rename_dict_regr, axis='columns')
leg_fs_test= leg_fs_test.rename(rename_dict_type)
leg_fs_test= leg_fs_test.loc[row_order, col_order]

leg_no_fs['diff']= (leg_no_fs['y_pred_1'] - leg_no_fs['y_test_1'])**2
leg_no_fs_test= leg_no_fs.pivot(index='type', columns='model', values='diff')
leg_no_fs_test= leg_no_fs_test.rename(rename_dict_regr, axis='columns')
leg_no_fs_test= leg_no_fs_test.rename(rename_dict_type)
leg_no_fs_test= leg_no_fs_test.loc[row_order, col_order]

In [12]:
leg_ttest_latex= test_compare(leg_fs_test, leg_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(leg_ttest_latex)

#save results to tex file
with open(path_leg_ttest_latex, 'w') as f:
    f.write(latex_template.format(leg_ttest_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
multi-atlas & 3e-45 & 3e-55 & 3e-51 & 7e-68 & 1e-59 \\
mean & 1e-50 & 4e-37 & 2e-18 & 1e-18 & 4e-59 \\
atlas 3 & 8e-16 & 2e-37 & 1e-24 & 5e-43 & 1e-45 \\
atlas 4 & 8e-31 & 7e-22 & 3e-15 & 2e-12 & 1e-31 \\
atlas 5 & 5e-51 & 6e-23 & 3e-22 & 9e-27 & 8e-49 \\
atlas 6 & 1e-43 & 9e-14 & 2e-16 & 1e-23 & 2e-60 \\
atlas 7 & 6e-59 & 1e-21 & 6e-13 & 4e-28 & 7e-78 \\
\bottomrule
\end{tabular}



In [13]:
leg_fs_ttest_diff_latex= test_diff(leg_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(leg_fs_ttest_diff_latex)

#save results to tex file
with open(path_leg_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(leg_fs_ttest_diff_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
mean & 3e-35 & 2e-17 & 7e-31 & 6e-21 & 7e-07 \\
atlas 3 & 1e-26 & 6e-13 & 8e-20 & 7e-14 & 6e-01 \\
atlas 4 & 4e-29 & 2e-22 & 1e-33 & 3e-36 & 9e-17 \\
atlas 5 & 9e-41 & 4e-32 & 8e-34 & 2e-33 & 2e-04 \\
atlas 6 & 3e-39 & 8e-27 & 4e-39 & 1e-34 & 5e-15 \\
atlas 7 & 1e-25 & 5e-34 & 4e-36 & 1e-33 & 1e-05 \\
\bottomrule
\end{tabular}



In [14]:
leg_fs_ttest_diff_cross_latex= test_diff_cross(leg_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(leg_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_leg_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(leg_fs_ttest_diff_cross_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
model &  &  &  &  &  \\
\midrule
linear & nan & 1e-06 & 5e-01 & 7e-03 & 5e-25 \\
PLS & 1e-06 & nan & 5e-07 & 1e-02 & 2e-23 \\
lasso & 5e-01 & 5e-07 & nan & 5e-03 & 7e-26 \\
ridge & 7e-03 & 1e-02 & 5e-03 & nan & 3e-24 \\
kNN & 5e-25 & 2e-23 & 7e-26 & 3e-24 & nan \\
\bottomrule
\end{tabular}



## loin $r^2$ results

In [15]:
loin_fs_results= loin_fs.pivot(index='type', columns='model', values='r2_1')
loin_fs_results= loin_fs_results.rename(rename_dict_regr, axis='columns')
loin_fs_results= loin_fs_results.rename(rename_dict_type)
loin_fs_results= loin_fs_results.apply(lambda x: np.round(x, 4))
loin_fs_results= loin_fs_results.loc[row_order, col_order]

loin_no_fs_results= loin_no_fs.pivot(index='type', columns='model', values='r2_1')
loin_no_fs_results= loin_no_fs_results.rename(rename_dict_regr, axis='columns')
loin_no_fs_results= loin_no_fs_results.rename(rename_dict_type)
loin_no_fs_results= loin_no_fs_results.apply(lambda x: np.round(x, 4))
loin_no_fs_results= loin_no_fs_results.loc[row_order, col_order]

loin_all= pd.concat([loin_fs_results, loin_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
loin_all.columns= columns

In [16]:
loin_all_r2_latex= loin_all.to_latex()
print(loin_all_r2_latex)

#save results to tex file
with open(path_loin_r2_latex, 'w') as f:
    f.write(latex_template.format(loin_all_r2_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
multi-atlas & 0.972600 & 0.969900 & 0.952400 & 0.992900 & 0.894800 & 0.736900 & 0.904600 & 0.903400 & 0.909000 & 0.761900 \\
mean & 0.939400 & 0.939400 & 0.939400 & 0.939400 & 0.887900 & 0.811000 & 0.910500 & 0.919300 & 0.913800 & 0.773700 \\
atlas 3 & 0.929600 & 0.934900 & 0.934900 & 0.934900 & 0.866400 & 0.805100 & 0.911500 & 0.917500 & 0.909900 & 0.729900 \\
atlas 4 & 0.942500 & 0.940300 & 0.942500 & 0.942600 & 0.896200 & 0.840600 & 0.927300 & 0.931800 & 0.927200 & 0.748400 \\
atlas 5 & 0.926700 & 0.932500 & 0.931900 & 0.931100 & 0.873800 & 0.860700 & 0.906200 & 0.903900 & 0.902800 & 0.685100 \\
atlas 6 & 0.940200 & 0.943100 & 0.944400 & 0.939500 & 0.879900 & 0.857300 & 0.910600 & 0.920800 & 0.917800 & 0.717900 \\
atlas 7 & 0.941200 & 0.

## loin RMSE results

In [17]:
loin_fs_results_rmse= loin_fs.pivot(index='type', columns='model', values='rmse_1')
loin_fs_results_rmse= loin_fs_results_rmse.rename(rename_dict_regr, axis='columns')
loin_fs_results_rmse= loin_fs_results_rmse.rename(rename_dict_type)
loin_fs_results_rmse= loin_fs_results_rmse.apply(lambda x: np.round(x, 4))
loin_fs_results_rmse= loin_fs_results_rmse.loc[row_order, col_order]

loin_no_fs_results_rmse= loin_no_fs.pivot(index='type', columns='model', values='rmse_1')
loin_no_fs_results_rmse= loin_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
loin_no_fs_results_rmse= loin_no_fs_results_rmse.rename(rename_dict_type)
loin_no_fs_results_rmse= loin_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
loin_no_fs_results_rmse= loin_no_fs_results_rmse.loc[row_order, col_order]

loin_all_rmse= pd.concat([loin_fs_results_rmse, loin_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
loin_all_rmse.columns= columns

In [18]:
loin_all_rmse_latex= loin_all_rmse.to_latex()
print(loin_all_rmse_latex)

#save results to tex file
with open(path_loin_rmse_latex, 'w') as f:
    f.write(latex_template.format(loin_all_rmse_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
multi-atlas & 176.813900 & 185.331900 & 232.941700 & 89.656900 & 346.299000 & 547.572100 & 329.710400 & 331.708900 & 322.072300 & 520.913700 \\
mean & 262.815600 & 262.815600 & 262.800400 & 262.786400 & 357.469800 & 464.120700 & 319.347400 & 303.268500 & 313.444000 & 507.820000 \\
atlas 3 & 283.330700 & 272.425200 & 272.451600 & 272.400000 & 390.119500 & 471.279900 & 317.503100 & 306.590100 & 320.386900 & 554.815000 \\
atlas 4 & 255.887400 & 260.910400 & 255.888100 & 255.829000 & 343.931500 & 426.208500 & 287.869900 & 278.843500 & 287.941200 & 535.395300 \\
atlas 5 & 289.088000 & 277.309900 & 278.531800 & 280.135400 & 379.190900 & 398.423800 & 326.929300 & 330.987500 & 332.725400 & 598.990900 \\
atlas 6 & 261.131300 & 254.547600 & 251.66980

## t-test loin

In [19]:
loin_fs['diff']= (loin_fs['y_pred_1'] - loin_fs['y_test_1'])**2
loin_fs_test= loin_fs.pivot(index='type', columns='model', values='diff')
loin_fs_test= loin_fs_test.rename(rename_dict_regr, axis='columns')
loin_fs_test= loin_fs_test.rename(rename_dict_type)
loin_fs_test= loin_fs_test.loc[row_order, col_order]

loin_no_fs['diff']= (loin_no_fs['y_pred_1'] - loin_no_fs['y_test_1'])**2
loin_no_fs_test= loin_no_fs.pivot(index='type', columns='model', values='diff')
loin_no_fs_test= loin_no_fs_test.rename(rename_dict_regr, axis='columns')
loin_no_fs_test= loin_no_fs_test.rename(rename_dict_type)
loin_no_fs_test= loin_no_fs_test.loc[row_order, col_order]

In [20]:
loin_ttest_latex= test_compare(loin_fs_test, loin_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(loin_ttest_latex)

#save results to tex file
with open(path_loin_ttest_latex, 'w') as f:
    f.write(latex_template.format(loin_ttest_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
multi-atlas & 7e-77 & 4e-55 & 3e-51 & 3e-92 & 3e-59 \\
mean & 2e-56 & 7e-24 & 9e-29 & 7e-19 & 2e-45 \\
atlas 3 & 2e-58 & 8e-18 & 3e-21 & 2e-13 & 3e-31 \\
atlas 4 & 2e-53 & 2e-10 & 2e-08 & 3e-10 & 4e-56 \\
atlas 5 & 3e-25 & 6e-19 & 2e-17 & 6e-14 & 4e-39 \\
atlas 6 & 4e-47 & 2e-31 & 1e-37 & 1e-20 & 9e-60 \\
atlas 7 & 3e-52 & 1e-19 & 6e-36 & 1e-21 & 2e-13 \\
\bottomrule
\end{tabular}



In [21]:
loin_fs_ttest_diff_latex= test_diff(loin_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(loin_fs_ttest_diff_latex)

#save results to tex file
with open(path_loin_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(loin_fs_ttest_diff_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
mean & 5e-37 & 1e-35 & 2e-11 & 5e-93 & 3e-01 \\
atlas 3 & 9e-45 & 3e-39 & 3e-20 & 6e-94 & 1e-07 \\
atlas 4 & 8e-36 & 2e-28 & 6e-06 & 4e-90 & 7e-01 \\
atlas 5 & 9e-54 & 3e-47 & 4e-19 & 1e-99 & 3e-06 \\
atlas 6 & 8e-34 & 5e-26 & 2e-04 & 4e-58 & 5e-04 \\
atlas 7 & 5e-30 & 9e-29 & 4e-07 & 3e-76 & 3e-10 \\
\bottomrule
\end{tabular}



In [22]:
loin_fs_ttest_diff_cross_latex= test_diff_cross(loin_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(loin_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_loin_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(loin_fs_ttest_diff_cross_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
model &  &  &  &  &  \\
\midrule
linear & nan & 7e-02 & 1e-22 & 1e-72 & 2e-46 \\
PLS & 7e-02 & nan & 9e-17 & 2e-57 & 4e-44 \\
lasso & 1e-22 & 9e-17 & nan & 3e-84 & 3e-25 \\
ridge & 1e-72 & 2e-57 & 3e-84 & nan & 2e-70 \\
kNN & 2e-46 & 4e-44 & 3e-25 & 2e-70 & nan \\
\bottomrule
\end{tabular}



## belly $r^2$ results

In [23]:
belly_fs_results= belly_fs.pivot(index='type', columns='model', values='r2_1')
belly_fs_results= belly_fs_results.rename(rename_dict_regr, axis='columns')
belly_fs_results= belly_fs_results.rename(rename_dict_type)
belly_fs_results= belly_fs_results.apply(lambda x: np.round(x, 4))
belly_fs_results= belly_fs_results.loc[row_order, col_order]

belly_no_fs_results= belly_no_fs.pivot(index='type', columns='model', values='r2_1')
belly_no_fs_results= belly_no_fs_results.rename(rename_dict_regr, axis='columns')
belly_no_fs_results= belly_no_fs_results.rename(rename_dict_type)
belly_no_fs_results= belly_no_fs_results.apply(lambda x: np.round(x, 4))
belly_no_fs_results= belly_no_fs_results.loc[row_order, col_order]

belly_all= pd.concat([belly_fs_results, belly_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
belly_all.columns= columns

In [24]:
belly_all_r2_latex= belly_all.to_latex()
print(belly_all_r2_latex)

#save results to tex file
with open(path_belly_r2_latex, 'w') as f:
    f.write(latex_template.format(belly_all_r2_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
multi-atlas & 0.957900 & 0.972100 & 0.962000 & 0.972200 & 0.910000 & 0.685800 & 0.905800 & 0.914500 & 0.882000 & 0.764700 \\
mean & 0.930400 & 0.932700 & 0.932600 & 0.930800 & 0.880900 & 0.759200 & 0.902900 & 0.919200 & 0.907100 & 0.734800 \\
atlas 3 & 0.904500 & 0.905400 & 0.904900 & 0.904500 & 0.862700 & 0.706800 & 0.859100 & 0.880700 & 0.860000 & 0.721800 \\
atlas 4 & 0.939000 & 0.940500 & 0.935500 & 0.940300 & 0.908700 & 0.837500 & 0.908200 & 0.916300 & 0.916300 & 0.734700 \\
atlas 5 & 0.933400 & 0.936100 & 0.937200 & 0.933500 & 0.892000 & 0.813500 & 0.912700 & 0.922400 & 0.914500 & 0.661200 \\
atlas 6 & 0.924100 & 0.925300 & 0.924300 & 0.921700 & 0.863800 & 0.700300 & 0.901600 & 0.904600 & 0.903100 & 0.738700 \\
atlas 7 & 0.912400 & 0.

## belly RMSE results

In [25]:
belly_fs_results_rmse= belly_fs.pivot(index='type', columns='model', values='rmse_1')
belly_fs_results_rmse= belly_fs_results_rmse.rename(rename_dict_regr, axis='columns')
belly_fs_results_rmse= belly_fs_results_rmse.rename(rename_dict_type)
belly_fs_results_rmse= belly_fs_results_rmse.apply(lambda x: np.round(x, 4))
belly_fs_results_rmse= belly_fs_results_rmse.loc[row_order, col_order]

belly_no_fs_results_rmse= belly_no_fs.pivot(index='type', columns='model', values='rmse_1')
belly_no_fs_results_rmse= belly_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
belly_no_fs_results_rmse= belly_no_fs_results_rmse.rename(rename_dict_type)
belly_no_fs_results_rmse= belly_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
belly_no_fs_results_rmse= belly_no_fs_results_rmse.loc[row_order, col_order]

belly_all_rmse= pd.concat([belly_fs_results_rmse, belly_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
belly_all_rmse.columns= columns

In [26]:
belly_all_rmse_latex= belly_all_rmse.to_latex()
print(belly_all_rmse_latex)

#save results to tex file
with open(path_belly_rmse_latex, 'w') as f:
    f.write(latex_template.format(belly_all_rmse_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
multi-atlas & 115.235500 & 93.807800 & 109.418800 & 93.509500 & 168.396200 & 314.637500 & 172.267600 & 164.112800 & 192.842600 & 272.301100 \\
mean & 148.064000 & 145.597500 & 145.704000 & 147.660400 & 193.745100 & 275.421000 & 174.862500 & 159.559100 & 171.116700 & 289.065500 \\
atlas 3 & 173.434500 & 172.600800 & 173.100600 & 173.421300 & 207.980500 & 303.914800 & 210.696900 & 193.893300 & 210.037200 & 296.049900 \\
atlas 4 & 138.651500 & 136.883000 & 142.514800 & 137.137500 & 169.629200 & 226.261800 & 170.035200 & 162.405700 & 162.348200 & 289.119200 \\
atlas 5 & 144.846900 & 141.899700 & 140.615200 & 144.782400 & 184.470200 & 242.372100 & 165.883500 & 156.355700 & 164.137200 & 326.722400 \\
atlas 6 & 154.634000 & 153.390700 & 154.392300

## t-test belly

In [27]:
belly_fs['diff']= (belly_fs['y_pred_1'] - belly_fs['y_test_1'])**2
belly_fs_test= belly_fs.pivot(index='type', columns='model', values='diff')
belly_fs_test= belly_fs_test.rename(rename_dict_regr, axis='columns')
belly_fs_test= belly_fs_test.rename(rename_dict_type)
belly_fs_test= belly_fs_test.loc[row_order, col_order]

belly_no_fs['diff']= (belly_no_fs['y_pred_1'] - belly_no_fs['y_test_1'])**2
belly_no_fs_test= belly_no_fs.pivot(index='type', columns='model', values='diff')
belly_no_fs_test= belly_no_fs_test.rename(rename_dict_regr, axis='columns')
belly_no_fs_test= belly_no_fs_test.rename(rename_dict_type)
belly_no_fs_test= belly_no_fs_test.loc[row_order, col_order]

In [28]:
belly_ttest_latex= test_compare(belly_fs_test, belly_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(belly_ttest_latex)

#save results to tex file
with open(path_belly_ttest_latex, 'w') as f:
    f.write(latex_template.format(belly_ttest_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
multi-atlas & 1e-95 & 3e-53 & 2e-62 & 2e-100 & 2e-43 \\
mean & 5e-57 & 5e-27 & 2e-30 & 1e-23 & 3e-70 \\
atlas 3 & 6e-56 & 2e-18 & 1e-13 & 1e-21 & 5e-43 \\
atlas 4 & 5e-38 & 1e-24 & 1e-17 & 7e-25 & 6e-47 \\
atlas 5 & 7e-52 & 1e-26 & 2e-28 & 1e-23 & 5e-68 \\
atlas 6 & 8e-41 & 1e-20 & 3e-25 & 4e-20 & 7e-46 \\
atlas 7 & 5e-31 & 1e-08 & 2e-27 & 2e-23 & 4e-23 \\
\bottomrule
\end{tabular}



In [29]:
belly_fs_ttest_diff_latex= test_diff(belly_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(belly_fs_ttest_diff_latex)

#save results to tex file
with open(path_belly_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(belly_fs_ttest_diff_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
mean & 7e-21 & 4e-35 & 1e-25 & 3e-43 & 2e-15 \\
atlas 3 & 6e-36 & 4e-46 & 9e-44 & 9e-43 & 8e-14 \\
atlas 4 & 6e-15 & 1e-27 & 3e-21 & 2e-29 & 7e-01 \\
atlas 5 & 4e-15 & 3e-43 & 1e-17 & 4e-40 & 1e-04 \\
atlas 6 & 6e-42 & 1e-59 & 2e-57 & 1e-49 & 9e-19 \\
atlas 7 & 2e-28 & 5e-40 & 5e-33 & 3e-40 & 5e-14 \\
\bottomrule
\end{tabular}



In [30]:
belly_fs_ttest_diff_cross_latex= test_diff_cross(belly_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(belly_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_belly_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(belly_fs_ttest_diff_cross_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
model &  &  &  &  &  \\
\midrule
linear & nan & 5e-11 & 5e-04 & 6e-16 & 4e-27 \\
PLS & 5e-11 & nan & 2e-10 & 9e-01 & 2e-52 \\
lasso & 5e-04 & 2e-10 & nan & 1e-13 & 2e-35 \\
ridge & 6e-16 & 9e-01 & 1e-13 & nan & 5e-48 \\
kNN & 4e-27 & 2e-52 & 2e-35 & 5e-48 & nan \\
\bottomrule
\end{tabular}



## shoulder $r^2$ results

In [31]:
shoulder_fs_results= shoulder_fs.pivot(index='type', columns='model', values='r2_1')
shoulder_fs_results= shoulder_fs_results.rename(rename_dict_regr, axis='columns')
shoulder_fs_results= shoulder_fs_results.rename(rename_dict_type)
shoulder_fs_results= shoulder_fs_results.apply(lambda x: np.round(x, 4))
shoulder_fs_results= shoulder_fs_results.loc[row_order, col_order]

shoulder_no_fs_results= shoulder_no_fs.pivot(index='type', columns='model', values='r2_1')
shoulder_no_fs_results= shoulder_no_fs_results.rename(rename_dict_regr, axis='columns')
shoulder_no_fs_results= shoulder_no_fs_results.rename(rename_dict_type)
shoulder_no_fs_results= shoulder_no_fs_results.apply(lambda x: np.round(x, 4))
shoulder_no_fs_results= shoulder_no_fs_results.loc[row_order, col_order]

shoulder_all= pd.concat([shoulder_fs_results, shoulder_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
shoulder_all.columns= columns

In [32]:
shoulder_all_r2_latex= shoulder_all.to_latex()
print(shoulder_all_r2_latex)

#save results to tex file
with open(path_shoulder_r2_latex, 'w') as f:
    f.write(latex_template.format(shoulder_all_r2_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
multi-atlas & 0.970300 & 0.964900 & 0.998000 & 0.992600 & 0.890300 & 0.847900 & 0.910900 & 0.919200 & 0.907600 & 0.756000 \\
mean & 0.919500 & 0.930600 & 0.923000 & 0.909500 & 0.857500 & 0.851900 & 0.900600 & 0.911200 & 0.909800 & 0.757500 \\
atlas 3 & 0.937700 & 0.941800 & 0.939000 & 0.939600 & 0.871800 & 0.852000 & 0.916000 & 0.924200 & 0.922300 & 0.780800 \\
atlas 4 & 0.920100 & 0.933300 & 0.920200 & 0.920700 & 0.860100 & -2.494400 & 0.905100 & 0.899400 & 0.905900 & 0.694400 \\
atlas 5 & 0.941000 & 0.940200 & 0.941000 & 0.940700 & 0.838200 & 0.842800 & 0.873100 & 0.918700 & 0.892800 & 0.530700 \\
atlas 6 & 0.926700 & 0.912600 & 0.943100 & 0.942000 & 0.808500 & 0.832300 & 0.899400 & 0.914700 & 0.928700 & 0.655200 \\
atlas 7 & 0.912800 & 0

## shoulder RMSE results

In [33]:
shoulder_fs_results_rmse= shoulder_fs.pivot(index='type', columns='model', values='rmse_1')
shoulder_fs_results_rmse= shoulder_fs_results_rmse.rename(rename_dict_regr, axis='columns')
shoulder_fs_results_rmse= shoulder_fs_results_rmse.rename(rename_dict_type)
shoulder_fs_results_rmse= shoulder_fs_results_rmse.apply(lambda x: np.round(x, 4))
shoulder_fs_results_rmse= shoulder_fs_results_rmse.loc[row_order, col_order]

shoulder_no_fs_results_rmse= shoulder_no_fs.pivot(index='type', columns='model', values='rmse_1')
shoulder_no_fs_results_rmse= shoulder_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
shoulder_no_fs_results_rmse= shoulder_no_fs_results_rmse.rename(rename_dict_type)
shoulder_no_fs_results_rmse= shoulder_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
shoulder_no_fs_results_rmse= shoulder_no_fs_results_rmse.loc[row_order, col_order]

shoulder_all_rmse= pd.concat([shoulder_fs_results_rmse, shoulder_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
shoulder_all_rmse.columns= columns

In [34]:
shoulder_all_rmse_latex= shoulder_all_rmse.to_latex()
print(shoulder_all_rmse_latex)

#save results to tex file
with open(path_shoulder_rmse_latex, 'w') as f:
    f.write(latex_template.format(shoulder_all_rmse_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
multi-atlas & 124.531600 & 135.450700 & 32.502200 & 62.133500 & 239.471600 & 281.959700 & 215.718600 & 205.471200 & 219.782600 & 357.053500 \\
mean & 205.096900 & 190.365300 & 200.617100 & 217.438800 & 272.866700 & 278.217000 & 227.877700 & 215.354300 & 217.044800 & 355.940100 \\
atlas 3 & 180.414100 & 174.354000 & 178.502700 & 177.679800 & 258.827700 & 278.110600 & 209.538200 & 199.042500 & 201.453500 & 338.451900 \\
atlas 4 & 204.372200 & 186.633400 & 204.156400 & 203.560000 & 270.396200 & 1351.271000 & 222.709700 & 229.286800 & 221.742600 & 399.596700 \\
atlas 5 & 175.590900 & 176.800100 & 175.601500 & 176.068600 & 290.739900 & 286.639800 & 257.462300 & 206.155000 & 236.703300 & 495.180300 \\
atlas 6 & 195.678400 & 213.724800 & 172.48240

## t-test shoulder

In [35]:
shoulder_fs['diff']= (shoulder_fs['y_pred_1'] - shoulder_fs['y_test_1'])**2
shoulder_fs_test= shoulder_fs.pivot(index='type', columns='model', values='diff')
shoulder_fs_test= shoulder_fs_test.rename(rename_dict_regr, axis='columns')
shoulder_fs_test= shoulder_fs_test.rename(rename_dict_type)
shoulder_fs_test= shoulder_fs_test.loc[row_order, col_order]

shoulder_no_fs['diff']= (shoulder_no_fs['y_pred_1'] - shoulder_no_fs['y_test_1'])**2
shoulder_no_fs_test= shoulder_no_fs.pivot(index='type', columns='model', values='diff')
shoulder_no_fs_test= shoulder_no_fs_test.rename(rename_dict_regr, axis='columns')
shoulder_no_fs_test= shoulder_no_fs_test.rename(rename_dict_type)
shoulder_no_fs_test= shoulder_no_fs_test.loc[row_order, col_order]

In [36]:
shoulder_ttest_latex= test_compare(shoulder_fs_test, shoulder_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(shoulder_ttest_latex)

#save results to tex file
with open(path_shoulder_ttest_latex, 'w') as f:
    f.write(latex_template.format(shoulder_ttest_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
multi-atlas & 2e-60 & 1e-86 & 6e-97 & 1e-96 & 6e-46 \\
mean & 1e-22 & 2e-16 & 6e-06 & 9e-01 & 5e-19 \\
atlas 3 & 1e-33 & 6e-25 & 5e-12 & 8e-15 & 6e-33 \\
atlas 4 & 5e-06 & 1e-26 & 2e-14 & 3e-07 & 7e-45 \\
atlas 5 & 2e-34 & 1e-57 & 5e-27 & 4e-36 & 1e-70 \\
atlas 6 & 3e-10 & 2e-04 & 2e-35 & 1e-22 & 2e-23 \\
atlas 7 & 1e-29 & 2e-45 & 4e-20 & 4e-48 & 2e-34 \\
\bottomrule
\end{tabular}



In [37]:
shoulder_fs_ttest_diff_latex= test_diff(shoulder_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(shoulder_fs_ttest_diff_latex)

#save results to tex file
with open(path_shoulder_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(shoulder_fs_ttest_diff_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
mean & 2e-46 & 1e-41 & 4e-116 & 3e-64 & 3e-08 \\
atlas 3 & 2e-37 & 4e-28 & 2e-95 & 5e-81 & 2e-04 \\
atlas 4 & 5e-49 & 8e-43 & 1e-110 & 8e-95 & 2e-06 \\
atlas 5 & 5e-21 & 8e-29 & 7e-82 & 3e-69 & 1e-10 \\
atlas 6 & 2e-57 & 5e-41 & 4e-76 & 2e-62 & 5e-16 \\
atlas 7 & 7e-51 & 5e-55 & 1e-100 & 5e-86 & 5e-43 \\
\bottomrule
\end{tabular}



In [38]:
shoulder_fs_ttest_diff_cross_latex= test_diff_cross(shoulder_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(shoulder_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_shoulder_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(shoulder_fs_ttest_diff_cross_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
model &  &  &  &  &  \\
\midrule
linear & nan & 4e-04 & 3e-74 & 1e-47 & 2e-36 \\
PLS & 4e-04 & nan & 3e-93 & 4e-64 & 3e-33 \\
lasso & 3e-74 & 3e-93 & nan & 5e-69 & 7e-65 \\
ridge & 1e-47 & 4e-64 & 5e-69 & nan & 4e-59 \\
kNN & 2e-36 & 3e-33 & 7e-65 & 4e-59 & nan \\
\bottomrule
\end{tabular}



## body $r^2$ results

In [39]:
body_fs_results= body_fs.pivot(index='type', columns='model', values='r2_1')
body_fs_results= body_fs_results.rename(rename_dict_regr, axis='columns')
body_fs_results= body_fs_results.rename(rename_dict_type)
body_fs_results= body_fs_results.apply(lambda x: np.round(x, 4))
body_fs_results= body_fs_results.loc[row_order, col_order]

body_no_fs_results= body_no_fs.pivot(index='type', columns='model', values='r2_1')
body_no_fs_results= body_no_fs_results.rename(rename_dict_regr, axis='columns')
body_no_fs_results= body_no_fs_results.rename(rename_dict_type)
body_no_fs_results= body_no_fs_results.apply(lambda x: np.round(x, 4))
body_no_fs_results= body_no_fs_results.loc[row_order, col_order]

body_all= pd.concat([body_fs_results, body_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
body_all.columns= columns

In [40]:
body_all_r2_latex= body_all.to_latex()
print(body_all_r2_latex)

#save results to tex file
with open(path_body_r2_latex, 'w') as f:
    f.write(latex_template.format(body_all_r2_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
multi-atlas & 0.990600 & 0.989500 & 0.989100 & 0.989500 & 0.912900 & 0.889800 & 0.973900 & 0.970400 & 0.972400 & 0.571500 \\
mean & 0.985100 & 0.984900 & 0.984300 & 0.985300 & 0.903100 & 0.405600 & 0.972700 & 0.975300 & 0.973400 & 0.626300 \\
atlas 3 & 0.981700 & 0.981800 & 0.984700 & 0.985300 & 0.876500 & -0.433300 & 0.967400 & 0.966400 & 0.964700 & 0.634800 \\
atlas 4 & 0.982200 & 0.986800 & 0.982600 & 0.986800 & 0.871200 & -2.591900 & 0.970200 & 0.969100 & 0.965600 & 0.560000 \\
atlas 5 & 0.985500 & 0.984300 & 0.985500 & 0.986900 & 0.899900 & -0.051300 & 0.973800 & 0.972900 & 0.969100 & 0.488800 \\
atlas 6 & 0.986700 & 0.986700 & 0.986300 & 0.987000 & 0.835300 & -1.328200 & 0.980500 & 0.975900 & 0.976700 & 0.576000 \\
atlas 7 & 0.980300 

## body RMSE results

In [41]:
body_fs_results_rmse= body_fs.pivot(index='type', columns='model', values='rmse_1')
body_fs_results_rmse= body_fs_results_rmse.rename(rename_dict_regr, axis='columns')
body_fs_results_rmse= body_fs_results_rmse.rename(rename_dict_type)
body_fs_results_rmse= body_fs_results_rmse.apply(lambda x: np.round(x, 4))
body_fs_results_rmse= body_fs_results_rmse.loc[row_order, col_order]

body_no_fs_results_rmse= body_no_fs.pivot(index='type', columns='model', values='rmse_1')
body_no_fs_results_rmse= body_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
body_no_fs_results_rmse= body_no_fs_results_rmse.rename(rename_dict_type)
body_no_fs_results_rmse= body_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
body_no_fs_results_rmse= body_no_fs_results_rmse.loc[row_order, col_order]

body_all_rmse= pd.concat([body_fs_results_rmse, body_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
body_all_rmse.columns= columns

In [42]:
body_all_rmse_latex= body_all_rmse.to_latex()
print(body_all_rmse_latex)

#save results to tex file
with open(path_body_rmse_latex, 'w') as f:
    f.write(latex_template.format(body_all_rmse_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
multi-atlas & 474.682200 & 501.340400 & 511.612700 & 501.861500 & 1444.975200 & 1624.908500 & 790.537100 & 842.835900 & 812.665800 & 3204.586400 \\
mean & 597.210400 & 602.126800 & 612.741700 & 593.828100 & 1523.551200 & 3773.943100 & 808.548000 & 768.657900 & 798.724000 & 2992.346900 \\
atlas 3 & 661.466100 & 659.600600 & 604.777700 & 593.934300 & 1720.604600 & 5860.615300 & 883.740300 & 897.855700 & 920.319100 & 2958.314500 \\
atlas 4 & 653.540100 & 562.305200 & 645.426600 & 563.141400 & 1757.136000 & 9277.586900 & 844.782400 & 860.812300 & 907.543600 & 3247.138800 \\
atlas 5 & 590.367800 & 613.242100 & 590.367900 & 560.494900 & 1549.098900 & 5019.121900 & 793.082700 & 805.468700 & 860.195900 & 3500.162500 \\
atlas 6 & 565.050800 & 564.69

## t-test body

In [43]:
body_fs['diff']= (body_fs['y_pred_1'] - body_fs['y_test_1'])**2
body_fs_test= body_fs.pivot(index='type', columns='model', values='diff')
body_fs_test= body_fs_test.rename(rename_dict_regr, axis='columns')
body_fs_test= body_fs_test.rename(rename_dict_type)
body_fs_test= body_fs_test.loc[row_order, col_order]

body_no_fs['diff']= (body_no_fs['y_pred_1'] - body_no_fs['y_test_1'])**2
body_no_fs_test= body_no_fs.pivot(index='type', columns='model', values='diff')
body_no_fs_test= body_no_fs_test.rename(rename_dict_regr, axis='columns')
body_no_fs_test= body_no_fs_test.rename(rename_dict_type)
body_no_fs_test= body_no_fs_test.loc[row_order, col_order]

In [44]:
body_ttest_latex= test_compare(body_fs_test, body_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(body_ttest_latex)

#save results to tex file
with open(path_body_ttest_latex, 'w') as f:
    f.write(latex_template.format(body_ttest_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
multi-atlas & 1e-77 & 1e-59 & 1e-55 & 2e-45 & 1e-32 \\
mean & 2e-30 & 5e-28 & 5e-38 & 6e-31 & 1e-29 \\
atlas 3 & 2e-21 & 2e-53 & 2e-37 & 3e-66 & 2e-28 \\
atlas 4 & 4e-09 & 1e-45 & 9e-48 & 3e-55 & 6e-34 \\
atlas 5 & 1e-03 & 1e-30 & 2e-70 & 7e-62 & 1e-35 \\
atlas 6 & 6e-08 & 6e-15 & 4e-44 & 1e-23 & 5e-32 \\
atlas 7 & 3e-15 & 2e-30 & 1e-46 & 6e-31 & 1e-34 \\
\bottomrule
\end{tabular}



In [45]:
body_fs_ttest_diff_latex= test_diff(body_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(body_fs_ttest_diff_latex)

#save results to tex file
with open(path_body_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(body_fs_ttest_diff_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
mean & 3e-23 & 7e-20 & 2e-21 & 1e-11 & 7e-03 \\
atlas 3 & 3e-39 & 1e-34 & 3e-15 & 4e-15 & 3e-11 \\
atlas 4 & 1e-34 & 1e-06 & 2e-23 & 2e-08 & 1e-16 \\
atlas 5 & 2e-22 & 2e-27 & 3e-17 & 2e-06 & 1e-04 \\
atlas 6 & 8e-15 & 6e-13 & 9e-12 & 2e-07 & 1e-29 \\
atlas 7 & 8e-53 & 3e-36 & 4e-47 & 6e-39 & 3e-17 \\
\bottomrule
\end{tabular}



In [46]:
body_fs_ttest_diff_cross_latex= test_diff_cross(body_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(body_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_body_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(body_fs_ttest_diff_cross_latex))

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
model &  &  &  &  &  \\
\midrule
linear & nan & 2e-03 & 2e-05 & 9e-03 & 2e-43 \\
PLS & 2e-03 & nan & 1e-01 & 1e+00 & 5e-42 \\
lasso & 2e-05 & 1e-01 & nan & 3e-01 & 5e-42 \\
ridge & 9e-03 & 1e+00 & 3e-01 & nan & 3e-42 \\
kNN & 2e-43 & 5e-42 & 5e-42 & 3e-42 & nan \\
\bottomrule
\end{tabular}



### Convert tex files to pdfs

In [47]:
import shutil
import glob

pdflatex_path= shutil.which("pdflatex")
if path_prefix_results != os.path.basename(os.getcwd()):
    os.chdir(path_prefix_results)
texs= glob.glob("*tex")
temp = "tmp"
os.makedirs(temp, exist_ok=True)

In [48]:
for tex in texs:
    print(f"Converting {tex}...")
    cmd= f"{pdflatex_path} {tex} >> /dev/null"
    os.system(cmd)
    for i in glob.glob("*.aux") + glob.glob("*.log"):
        shutil.move(os.path.abspath(i), os.path.join(os.path.abspath(temp),i))

Converting belly_fs_ttest_diff_cross.tex...
Converting loin_r2.tex...
Converting leg_fs_ttest_diff_cross.tex...
Converting belly_fs_ttest_diff.tex...
Converting leg_fs_ttest_diff.tex...
Converting shoulder_fs_ttest_diff.tex...
Converting shoulder_fs_ttest_diff_cross.tex...
Converting body_ttest.tex...
Converting body_fs_ttest_diff.tex...
Converting body_rmse.tex...
Converting belly_ttest.tex...
Converting loin_rmse.tex...
Converting loin_fs_ttest_diff_cross.tex...
Converting shoulder_r2.tex...
Converting shoulder_ttest.tex...
Converting leg_rmse.tex...
Converting leg_ttest.tex...
Converting shoulder_rmse.tex...
Converting leg_r2.tex...
Converting body_fs_ttest_diff_cross.tex...
Converting loin_fs_ttest_diff.tex...
Converting belly_rmse.tex...
Converting loin_ttest.tex...
Converting body_r2.tex...
Converting belly_r2.tex...
